In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import csv
import math
import random
import time

In [2]:
def alpha_degree(graph,k):
  start=time.time()
  degrees=[]
  triangles=[]
  sorted_nodes = sorted(graph.degree, key=lambda x: x[1], reverse=True)
  num_nodes = len(sorted_nodes)
  third = num_nodes // 3  # Integer division to get roughly equal parts
  list1 = sorted_nodes[:third]
  list2 = sorted_nodes[third:2 * third]
  list3 = sorted_nodes[2 * third:]
  sample_size=math.floor(k*third)
  node_sample_1 = random.sample(list1, sample_size)
  node_sample_2=random.sample(list2, sample_size)
  node_sample_3=random.sample(list3,sample_size)
  node_samples=node_sample_1+node_sample_2+node_sample_3
  for node,degree in node_samples:
    degree=0 if degree<=1 else math.log(degree)
    triangle=nx.triangles(graph,node)
    triangle=0 if triangle<=1 else math.log(triangle)
    degrees.append(degree)
    triangles.append(triangle)

  degrees=np.array(degrees)
  triangles=np.array(triangles)
  b, a = np.polyfit(degrees, triangles, deg=1)
  end=time.time()
  return [b,end-start,a]


In [4]:
def exact_netx(graph):
  start = time.time()
  t = sum(nx.triangles(graph).values())/3
  end = time.time()
  return {"triangles": t, "time": end - start}

In [14]:
def variance_reduction(graph,sample_size):
 start = time.time()
 nodes=[]
 degrees=[]
 alpha_result=alpha_degree(graph,0.05)
 alpha=alpha_result[0]
 factor=math.e**alpha_result[2]
 degree_alpha_sum=0
 for node in graph:
   nodes.append(node)
   degree=graph.degree(node)**alpha
   degrees.append(degree)
   degree_alpha_sum+=degree
 n=len(nodes)
 node_sample = random.choices(nodes,weights=degrees,k=sample_size)
 sample_diff = 0
 for node in node_sample:
    sample_diff += (nx.triangles(graph, node)-factor*(graph.degree(node)**alpha))
 diff=(n/3*sample_size)*sample_diff
 t = diff+degree_alpha_sum*factor
 end=time.time()
 return {"triangles": t, "time": end - start}

In [16]:
# Sampling Testing Function
def testing_graph_methods_sampling(graph, accuracies, times,algo): 
    exact_netx_result = exact_netx(graph)
    exact_triangles = exact_netx_result['triangles']
    print(f"Exact NetworkX Triangles: {exact_triangles}, Time: {exact_netx_result['time']}s")
    total_nodes = len(graph.nodes())
    sample_sizes = [int(total_nodes * percentage / 100) for percentage in range(5,55,5) ]
    algo_name=algo.__name__
    method_accuracies=[0 for sample_size in range(0,len(sample_sizes)+1) ]
    method_times=[0 for percentage in range(0,len(sample_sizes)+1)]
 
# create for loop for trials and another for loop for sample size according to sampling; between 5% and 50% in increments of 5% f the sample size given.
    # evaluate each method with respective names according to the methods being used
    for trial in range(0,1):
            # Run 10 trials for the current method and sample size
            iteration=0
            for sample_size in sample_sizes:
                print(f"Trial {trial} for {algo_name} with Sample Size: {sample_size}")
                result = algo(graph, sample_size)
                rel_error = abs(exact_triangles - result['triangles']) / exact_triangles
                method_accuracies[iteration]+=rel_error
                method_times[iteration]+=result['time']
                iteration+=1
    #Go over each sample size and average their accuracies and times
    for iteration in range(0,10):   
      avg_time = method_times[iteration]
      avg_accuracy=method_accuracies[iteration]
      accuracies.append((algo_name, sample_sizes[iteration], avg_accuracy))
      times.append((algo_name, sample_sizes[iteration], avg_time))

accuracies = []
times = []
 
fb_graph = nx.read_edgelist('facebook_combined.txt', create_using=nx.Graph(), nodetype=int)
#print(variance_reduction(fb_graph,2000))
 
testing_graph_methods_sampling(fb_graph, accuracies, times,variance_reduction)
print("Accuracies:", accuracies)
print("Times:", times)
with open('output_accuracy.csv', 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(accuracies)
with open('output_time.csv', 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(times)

Exact NetworkX Triangles: 1612010.0, Time: 0.7527480125427246s
Trial 0 for variance_reduction with Sample Size: 201
Trial 0 for variance_reduction with Sample Size: 403
Trial 0 for variance_reduction with Sample Size: 605
Trial 0 for variance_reduction with Sample Size: 807
Trial 0 for variance_reduction with Sample Size: 1009
Trial 0 for variance_reduction with Sample Size: 1211
Trial 0 for variance_reduction with Sample Size: 1413
Trial 0 for variance_reduction with Sample Size: 1615
Trial 0 for variance_reduction with Sample Size: 1817
Trial 0 for variance_reduction with Sample Size: 2019
Accuracies: [('variance_reduction', 201, 438812.807635667), ('variance_reduction', 403, 1666917.5345392448), ('variance_reduction', 605, 6015833.692004554), ('variance_reduction', 807, 7127214.706403954), ('variance_reduction', 1009, 9772600.123060988), ('variance_reduction', 1211, 24830341.786878157), ('variance_reduction', 1413, 20199639.713722855), ('variance_reduction', 1615, 24408991.649078973